### Setup
#### PIP Install Packages and dependencies

In [ ]:
%pip install google-cloud-bigquery

In [ ]:
# Automatically restart kernel after installs
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True) 

### Set up GCP project
1. Select GCP project.
2. Enable billing.
3. Enable the AI Platform APIs and Compute Engine APIs.
4. Enter project ID and region.

In [1]:
PROJECT_ID = "bionic-union-378011" 
REGION = 'US'

### Import libraries and define constants

In [2]:
from google.cloud import bigquery
import pandas as pd

pd.set_option('display.float_format', lambda x: '%.3f' % x)

### Create a BigQuery dataset

In [4]:
DATASET_NAME = "bqtest"
!bq mk --location=$REGION --dataset $PROJECT_ID:$DATASET_NAME

Dataset 'bionic-union-378011:bqtest' successfully created.


In [5]:
%load_ext google.cloud.bigquery

### The dataset

In [ ]:
%%bigquery --project $PROJECT_ID

SELECT 
    *
FROM
  `firebase-public-project.analytics_153293282.events_*`
    
TABLESAMPLE SYSTEM (1 PERCENT)

In [ ]:
%%bigquery --project $PROJECT_ID

SELECT 
    COUNT(DISTINCT user_pseudo_id) as count_distinct_users,
    COUNT(event_timestamp) as count_events
FROM
  `firebase-public-project.analytics_153293282.events_*`

### Preparing the training data

#### Step 1: Identifying the label for each user

In [7]:
%%bigquery --project $PROJECT_ID 

CREATE OR REPLACE VIEW bqtest.returningusers AS (
  WITH firstlasttouch AS (
    SELECT
      user_pseudo_id,
      MIN(event_timestamp) AS user_first_engagement,
      MAX(event_timestamp) AS user_last_engagement
    FROM
      `firebase-public-project.analytics_153293282.events_*`
    WHERE event_name="user_engagement"
    GROUP BY
      user_pseudo_id

  )
  SELECT
    user_pseudo_id,
    user_first_engagement,
    user_last_engagement,
    EXTRACT(MONTH from TIMESTAMP_MICROS(user_first_engagement)) as month,
    EXTRACT(DAYOFYEAR from TIMESTAMP_MICROS(user_first_engagement)) as julianday,
    EXTRACT(DAYOFWEEK from TIMESTAMP_MICROS(user_first_engagement)) as dayofweek,

    (user_first_engagement + 86400000000) AS ts_24hr_after_first_engagement,

IF (user_last_engagement < (user_first_engagement + 7*86400000000),
    1,
    0 ) AS churned,

IF (user_last_engagement <= (user_first_engagement + 600000000),
    1,
    0 ) AS bounced,
  FROM
    firstlasttouch
  GROUP BY
    1,2,3
    );

SELECT 
  * 
FROM 
  bqtest.returningusers 
LIMIT 100;

Query is running:   0%|          |

Downloading:   0%|          |

,user_pseudo_id,user_first_engagement,user_last_engagement,month,julianday,dayofweek,ts_24hr_after_first_engagement,churned,bounced
0,0D17D0247F158FA3E40D21EFF46D1651,1530130443215000,1538603802832004,6,178,4,1530216843215000,0,0
1,8B38AE00504B75A9ABAA20843DF4976A,1535249657757008,1537493936454034,8,238,1,1535336057757008,0,0
2,2979D1B301F7F36C666AE78523223737,1535896312328003,1535896878031025,9,245,1,1535982712328003,1,1
3,2C3B1F799EFF5833CC3AF913FFBC83E5,1535873309622000,1535873309622000,9,245,1,1535959709622000,1,1
4,87331C2AC02608DB03191D5AA032B993,1535949740890000,1535949752447001,9,246,2,1536036140890000,1,1
...,...,...,...,...,...,...,...,...,...
95,53A5A4C271525BAA74298747E100C376,1528950638416011,1536559799746003,6,165,5,1529037038416011,0,0
96,898A388994F84492EC0082F7E46C290A,1530701692390004,1531481212363002,7,185,4,1530788092390004,0,0
97,FCCCA96FC69F9ABD2CE486E809665499,1530759618316001,1530760274527018,7,186,5,1530846018316001,1,0
98,79CB6ACC45415E09A25132C8284A43FA,1529696977215009,1530855258144006,6,173,6,1529783377215009,0,0


In [8]:
%%bigquery --project $PROJECT_ID

SELECT
    bounced,
    churned, 
    COUNT(churned) as count_users
FROM
    bqtest.returningusers
GROUP BY 1,2
ORDER BY bounced

Query is running:   0%|          |

Downloading:   0%|          |

,bounced,churned,count_users
0,0,1,3292
1,0,0,4739
2,1,1,5557


In [9]:
%%bigquery --project $PROJECT_ID

SELECT
    COUNTIF(churned=1)/COUNT(churned) as churn_rate
FROM
    bqtest.returningusers
WHERE bounced = 0

Query is running:   0%|          |

Downloading:   0%|          |

,churn_rate
0,0.410


#### Step 2. Extracting demographic data for each user

In [10]:
%%bigquery --project $PROJECT_ID

CREATE OR REPLACE VIEW bqtest.user_demographics AS (

  WITH first_values AS (
      SELECT
          user_pseudo_id,
          geo.country as country,
          device.operating_system as operating_system,
          device.language as language,
          ROW_NUMBER() OVER (PARTITION BY user_pseudo_id ORDER BY event_timestamp DESC) AS row_num
      FROM `firebase-public-project.analytics_153293282.events_*`
      WHERE event_name="user_engagement"
      )
  SELECT * EXCEPT (row_num)
  FROM first_values
  WHERE row_num = 1
  );

SELECT
  *
FROM
  bqtest.user_demographics
LIMIT 10

Query is running:   0%|          |

Downloading:   0%|          |

,user_pseudo_id,country,operating_system,language
0,02FD5F0EA7D59257A5BFB20627E60746,Jordan,ANDROID,ar-ae
1,039E07F8DF825DEE56F4301208D7F758,Belgium,IOS,nl-be
2,075EBEDBF28134BE41FB4E7D943F858C,United States,IOS,en-us
3,07D5951C19A83B2CB9C8BA9319B5A9FD,United States,ANDROID,en-us
4,08E6794EAC892997E8C3BB2323173641,United States,IOS,en-us
5,0B1A026DF1EE470C33B79A79EBEC965C,United States,IOS,ja-us
6,0F52329D86C8201D2921BE648006E647,United States,IOS,en-us
7,0F66D1F804DD087B999A4C74CA29B329,United States,None,en-us
8,1197B3BDC92908B4C3BD12B33CB66DD5,Japan,IOS,ja-jp
9,125363861AAA26BC5862E54D738A342F,United States,IOS,en-au


#### Step 3. Extracting behavioral data for each user

In [ ]:
%%bigquery --project $PROJECT_ID

SELECT
    event_name,
    COUNT(event_name) as event_count
FROM
    `firebase-public-project.analytics_153293282.events_*`
GROUP BY 1
ORDER BY
   event_count DESC

In [12]:
%%bigquery --project $PROJECT_ID

CREATE OR REPLACE VIEW bqtest.user_aggregate_behavior AS (
WITH
  events_first24hr AS (
    #select user data only from first 24 hr of using the app
    SELECT
      e.*
    FROM
      `firebase-public-project.analytics_153293282.events_*` e
    JOIN
      bqtest.returningusers r
    ON
      e.user_pseudo_id = r.user_pseudo_id
    WHERE
      e.event_timestamp <= r.ts_24hr_after_first_engagement
    )
SELECT
  user_pseudo_id,
  SUM(IF(event_name = 'user_engagement', 1, 0)) AS cnt_user_engagement,
  SUM(IF(event_name = 'level_start_quickplay', 1, 0)) AS cnt_level_start_quickplay,
  SUM(IF(event_name = 'level_end_quickplay', 1, 0)) AS cnt_level_end_quickplay,
  SUM(IF(event_name = 'level_complete_quickplay', 1, 0)) AS cnt_level_complete_quickplay,
  SUM(IF(event_name = 'level_reset_quickplay', 1, 0)) AS cnt_level_reset_quickplay,
  SUM(IF(event_name = 'post_score', 1, 0)) AS cnt_post_score,
  SUM(IF(event_name = 'spend_virtual_currency', 1, 0)) AS cnt_spend_virtual_currency,
  SUM(IF(event_name = 'ad_reward', 1, 0)) AS cnt_ad_reward,
  SUM(IF(event_name = 'challenge_a_friend', 1, 0)) AS cnt_challenge_a_friend,
  SUM(IF(event_name = 'completed_5_levels', 1, 0)) AS cnt_completed_5_levels,
  SUM(IF(event_name = 'use_extra_steps', 1, 0)) AS cnt_use_extra_steps,
FROM
  events_first24hr
GROUP BY
  1
  );

SELECT
  *
FROM
  bqtest.user_aggregate_behavior
LIMIT 10

Query is running:   0%|          |

Downloading:   0%|          |

,user_pseudo_id,cnt_user_engagement,cnt_level_start_quickplay,cnt_level_end_quickplay,cnt_level_complete_quickplay,cnt_level_reset_quickplay,cnt_post_score,cnt_spend_virtual_currency,cnt_ad_reward,cnt_challenge_a_friend,cnt_completed_5_levels,cnt_use_extra_steps
0,CD896265D2278DDEEDA74C938150F20E,9,4,3,3,0,3,0,0,0,0,0
1,32C3B9708AFE48E9C73681ECA6582192,1,0,0,0,0,0,0,0,0,0,0
2,10BF22F70000BE1DAFBF08E6B5893C71,2,0,0,0,0,0,0,0,0,0,0
3,7919E77A6FB786831A0B13C69F1EF6C5,1,1,0,0,0,0,0,0,0,0,0
4,C96C96A3BFA142C6B393183BDB2B9DDB,281,85,84,26,1,26,3,0,0,0,3
5,214980227844539E15ABC4C4F3A79C28,173,0,0,0,0,17,0,0,0,0,0
6,62BC44432D6014D99E9B6E1C08C015DD,14,0,0,0,0,5,0,0,0,1,0
7,200F0E8F2569DC89591D61CD20C86F9E,3,8,0,0,8,0,0,0,0,0,0
8,04C1B862AFA39703243166C625EBBD40,6,2,1,0,0,0,0,0,4,0,0
9,54E11DDF707E8B5C7EA5CC4FD48CED88,4,1,1,1,0,1,0,0,0,0,0


#### Step 4: Combining the label, demographic and behavioral data together as training data

In [13]:
%%bigquery --project $PROJECT_ID

CREATE OR REPLACE VIEW bqtest.train AS (
    
  SELECT
    dem.*,
    IFNULL(beh.cnt_user_engagement, 0) AS cnt_user_engagement,
    IFNULL(beh.cnt_level_start_quickplay, 0) AS cnt_level_start_quickplay,
    IFNULL(beh.cnt_level_end_quickplay, 0) AS cnt_level_end_quickplay,
    IFNULL(beh.cnt_level_complete_quickplay, 0) AS cnt_level_complete_quickplay,
    IFNULL(beh.cnt_level_reset_quickplay, 0) AS cnt_level_reset_quickplay,
    IFNULL(beh.cnt_post_score, 0) AS cnt_post_score,
    IFNULL(beh.cnt_spend_virtual_currency, 0) AS cnt_spend_virtual_currency,
    IFNULL(beh.cnt_ad_reward, 0) AS cnt_ad_reward,
    IFNULL(beh.cnt_challenge_a_friend, 0) AS cnt_challenge_a_friend,
    IFNULL(beh.cnt_completed_5_levels, 0) AS cnt_completed_5_levels,
    IFNULL(beh.cnt_use_extra_steps, 0) AS cnt_use_extra_steps,
    ret.user_first_engagement,
    ret.month,
    ret.julianday,
    ret.dayofweek,
    ret.churned
  FROM
    bqtest.returningusers ret
  LEFT OUTER JOIN
    bqtest.user_demographics dem
  ON 
    ret.user_pseudo_id = dem.user_pseudo_id
  LEFT OUTER JOIN 
    bqtest.user_aggregate_behavior beh
  ON
    ret.user_pseudo_id = beh.user_pseudo_id
  WHERE ret.bounced = 0
  );

SELECT
  *
FROM
  bqtest.train
LIMIT 5

Query is running:   0%|          |

Downloading:   0%|          |

,user_pseudo_id,country,operating_system,language,cnt_user_engagement,cnt_level_start_quickplay,cnt_level_end_quickplay,cnt_level_complete_quickplay,cnt_level_reset_quickplay,cnt_post_score,cnt_spend_virtual_currency,cnt_ad_reward,cnt_challenge_a_friend,cnt_completed_5_levels,cnt_use_extra_steps,user_first_engagement,month,julianday,dayofweek,churned
0,9D535D543FD07B8247DEADB4669FADE6,United States,ANDROID,en-us,41,16,11,5,3,5,0,0,0,0,0,1531154706915004,7,190,2,0
1,5217AB1A454DAED6243E1C9818BE6A20,United States,ANDROID,en-us,89,0,0,0,0,27,0,0,0,1,1,1529861523949001,6,175,1,0
2,8F288CAB6C6396741C17C83D5FC44C5F,Nepal,ANDROID,en-gb,64,29,9,5,17,9,8,0,0,0,8,1531466573591001,7,194,6,1
3,41CA0FA528C7227EA1E592DCC4733C40,United States,ANDROID,en-us,25,11,9,7,1,7,0,0,0,0,0,1528914965271010,6,164,4,0
4,7566596A1D6ACA781692A7A0B89B06EF,United States,ANDROID,en-us,8,1,1,0,0,0,0,0,0,0,0,1528843166489001,6,163,3,0


### Train a logistic regression model

%%bigquery --project $PROJECT_ID

CREATE OR REPLACE MODEL bqtest.churn_logreg

OPTIONS(
  MODEL_TYPE="LOGISTIC_REG",
  INPUT_LABEL_COLS=["churned"]
) AS

SELECT
  *
FROM
  bqtest.train

### Model Evaluation

%%bigquery --project $PROJECT_ID

SELECT
  *
FROM
  ML.EVALUATE(MODEL bqtest.churn_logreg)

%%bigquery --project $PROJECT_ID

SELECT
  expected_label,
  _0 AS predicted_0,
  _1 AS predicted_1
FROM
  ML.CONFUSION_MATRIX(MODEL bqtest.churn_logreg)